In [1]:
"""
This cell is auto-generated by Garden.
You can use this widget to edit your notebooks metadata. 
That way, the next time you run this notebook, Garden will start it with the same environment.
Any changes made to your notebook's metadata using the widget will be saved when the notebook is saved.

Notebook metadata fields:
   - Global Garden DOI: If specified, `garden-ai notebook publish` will add all entrypoints in this notebook to the Garden with the given DOI.
     If you want to specify a different Garden DOI for individual entrypoints, you can provide that entrypoint's
     'entrypoint' decorator with the optional 'garden_doi' argument. Providing the decorator with a DOI
     will override the Global DOI for that specific entrypoint.
   - Base image name: The name of the garden base image you want to start this notebook with.
     To see a list of the available Garden base images, see the dropdown menu under 'Base Image' below or 
     use 'garden-ai notebook list-premade-images'
   - Requirements: Any additional requirements that should be installed in this notebook's container.
     After making changes to your notebook's requirements, the widget will show a 'Install new requirements' button
     that installs the new requirements to the container and updates your local requirements file if one was provided.
"""

from garden_ai.notebook_metadata import display_metadata_widget
display_metadata_widget()

Output()

## Your Model 🌱Garden🌱 Execution Environment

Use this notebook to write a function that executes your model(s). Tag that function with the `@entrypoint` decorator.

Garden will take this notebook and build a container with it. When Garden executes your `@entrypoint`, it will be like like you have just run all the cells of this notebook once. So you can install libraries with `!pip install` and your function can use those libraries. You can also define helper functions and constants to use in your `@entrypoint`.

In [2]:
from garden_ai.model_connectors import create_connector
from garden_ai import EntrypointMetadata, entrypoint, entrypoint_test

In [3]:
import torch

### Model connectors

Model connectors let Garden import metadata about your model.
They also have a `stage` method that you can use to download your model weights.

In [4]:
my_hugging_face_repo = create_connector("https://huggingface.co/bengal1/RTS", revision="30457d4779884d47bcf6c6fe0ba153a5c75885d3")
my_hugging_face_repo

README.md:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/site-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


HFConnector(repo_url=Url('https://huggingface.co/bengal1/RTS'), repo_id='bengal1/RTS', branch='main', revision='30457d4779884d47bcf6c6fe0ba153a5c75885d3', local_dir=PosixPath('models/RTS'), enable_imports=True, metadata=ModelMetadata(model_identifier='bengal1/RTS', model_repository='huggingface.co', model_version='30457d4779884d47bcf6c6fe0ba153a5c75885d3'), readme='', model_dir='models')

### Entrypoint metadata


To publish your function, Garden needs metadata so that other users can discover it.
Edit this EntrypointMetadata object to describe your function.


In [5]:
rts_entrypoint_meta = EntrypointMetadata(
    title="Retrogressive Thaw Slumps",
    description="Simplified example of training and running inference on a toy model using the Retrogressive Thaw Slumps (RTS) dataset.",
    authors=["Ben Galewsky", "Chia-Yu Hsu"],
    tags=["pytorch", "tutorial"],
    requirements=[]
)

### Helper Functions

Define any helper functions you need and use them in the function you want to let people run remotely

In [6]:
def preprocess(image):
    from torchvision.io import read_image
    from torchvision.transforms import v2 as T
    transforms = []

    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    transform =  T.Compose(transforms)


    image = (255.0 * (image - image.min()) / (image.max() - image.min())).to(
        torch.uint8
    )
    image = image[:3, ...]
    transformed_image = transform(image)

    x = torch.unsqueeze(transformed_image, 0)

    return x # [:3, ...]


### Write your entrypoint function that will run remotely

The `@entrypoint` decorator makes this function available to run in your garden when you publish the notebook.
Download your model weights and call your model in this function.

In the decorator be sure to include:
- your entrypoint metadata,
- connectors for any models you're using,

You can add your entrypoint to a Garden in two different ways.

If you want all entrypoints in this notebook be added to one Garden, set the `NOTEBOOK_GLOBAL_DOI` in your notebooks metadata or by using the `--doi` argument for `garden-ai notebook start`

If you want to specify different Gardens for different entrypoints, provide each decorator with the optional `garden_doi` argument.

If you both set the `NOTEBOOK_GLOBAL_DOI` and are providing a decorator with a DOI, the entrypoint will ONLY be added to the Garden given to the decorator.

To see all the DOIs of your gardens, use `garden-ai garden list`

In [14]:
@entrypoint(metadata=rts_entrypoint_meta,  model_connectors=[my_hugging_face_repo], garden_doi="10.26311/5fb6-f950")
def identify_rts(image_url):
    import torch
    import requests
    import tempfile
    from torchvision.io import read_image

    download_path = my_hugging_face_repo.stage()
    model = torch.load(download_path + "/model.pth", map_location=torch.device('cpu'))

    response = requests.get(image_url)
    with tempfile.NamedTemporaryFile(delete=True, suffix='.jpg') as tmp_file:
        # Write the content to the temporary file
        tmp_file.write(response.content)
        tmp_file_path = tmp_file.name
        image = read_image(tmp_file_path)

        scaled_tensor = preprocess(image)
        with torch.no_grad():
            output = model(scaled_tensor)

    def filter_predictions(pred, score_threshold=0.5):
        keep = pred["scores"] > score_threshold
        return {k: v[keep] for k, v in pred.items()}

    return filter_predictions(output[0])

### Test your entrypoint function

Finally, make sure your `@entrypoint` works!
When Garden makes a container from your notebook, it runs all the cells in order and saves the notebook. Then users invoke your `@entrypoint` in the context of the notebook.

Note on testing: any test functions that call your entrypoint (like the one below) should be marked with `@entrypoint_test(<entrypoint_being_tested>)`. This is because calling an entrypoint typically causes side-effects (such as downloading your model weights to disk) that shouldn't be "baked in" to the environment of the final published notebook. 

Anything marked with `@entrypoint_test` won't be run at publication time, so you don't need to remember to comment out your test code before publishing. We'll also use `@entrypoint_test` functions as example code for others to see how your entrypoint expects to be called. 

In [12]:
@entrypoint_test(identify_rts)
def test_run_my_model():
    from torchvision.io import read_image
    import requests
    import tempfile

    sample_image_url = "https://github.com/cyber2a/Cyber2A-RTS-ToyModel/blob/main/data/images/valtest_nitze_008.jpg?raw=true"
    return identify_rts(sample_image_url)
import os
os.environ["GARDEN_SKIP_TESTS"] = 'True'
test_run_my_model()

In [15]:
sample_image_url = "https://github.com/cyber2a/Cyber2A-RTS-ToyModel/blob/main/data/images/valtest_yg_055.jpg?raw=true"
print(identify_rts(sample_image_url))


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

{'boxes': tensor([[ 69.4303, 134.2501, 111.9559, 173.3981]]), 'labels': tensor([1]), 'scores': tensor([0.6037]), 'masks': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])}
